In [1]:
%load_ext autoreload
%autoreload 2

import os
import time
import sys

from utilities2014 import *

sys.path.append('/home/yuncong/project/opencv-2.4.9/release/lib/python2.7/site-packages')
import cv2

from joblib import Parallel, delayed

from collections import defaultdict, Counter
from itertools import combinations, chain, product

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import cPickle as pickle

In [3]:
from enum import Enum

class PolygonType(Enum):
    CLOSED = 'closed'
    OPEN = 'open'
    TEXTURE = 'textured'
    TEXTURE_WITH_CONTOUR = 'texture with contour'
    DIRECTION = 'directionality'

In [4]:
thetas = np.linspace(-np.pi/4, np.pi/4, 9)
n_theta = len(thetas)
Rs = [np.array([[np.cos(theta), np.sin(theta)], [-np.sin(theta), np.cos(theta)]]) for theta in thetas]

In [5]:
landmark_indices = range(17)

In [6]:
vertices_name = {PolygonType.OPEN: 'boundary_vertices_global',
PolygonType.CLOSED: 'boundary_vertices_global',
PolygonType.DIRECTION: 'striation_points_global',
PolygonType.TEXTURE: 'textured_area_vertices_global',
PolygonType.TEXTURE_WITH_CONTOUR: 'textured_area_vertices_global'
}

In [18]:
landmark_polygons = {}
for lm_ind in landmark_indices:
    print lm_ind
    
    with open('/home/yuncong/csd395/landmark_descriptors/RS141_0001_yuncong_08212015001655_landmark_descriptors_%d.pkl'%lm_ind, 'r') as f:
        landmark_polygons[lm_ind] = pickle.load(f)[4]['polygons']
    
#     landmark_polygons[lm_ind] = [(p['type'], p[vertices_name[p['type']]]) for p in polygons]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16


In [7]:
colors = (np.loadtxt(os.environ['GORDON_REPO_DIR'] + '/visualization/100colors.txt', skiprows=1) * 255).astype(np.int)

In [19]:
dm = DataManager(generate_hierarchy=False, stack='RS141', resol='x5', section=1)
dm._load_image()
origin_on_canvas = dm.load_pipeline_result('originPosOnCanvas', 'npy')

In [27]:
grid_ys1, grid_xs1 = np.meshgrid(range(0,dm.image_height, 50), range(0,dm.image_width, 1), indexing='ij')
grid_ys2, grid_xs2 = np.meshgrid(range(0,dm.image_height, 1), range(0,dm.image_width, 50), indexing='ij')

grid_ys = np.r_[grid_ys1.flat, grid_ys2.flat].astype(np.int)
grid_xs = np.r_[grid_xs1.flat, grid_xs2.flat].astype(np.int)
grids = np.c_[grid_xs.flat, grid_ys.flat]

np.save(os.environ['GORDON_RESULT_DIR'] + '/atlas_grids.npy', grids)

In [35]:
import cv2

def visualize_atlas(landmark_polygon_vertices_global):
    '''
    Visualize the atlas with landmarks in original orientation
    '''
    
    canvas_size = np.load(os.environ['GORDON_RESULT_DIR'] + '/paddedSize.npy')
    
    viz = 255 * np.ones(np.r_[canvas_size, 3], np.uint8)
    
    for x,y in grids + origin_on_canvas:
        cv2.circle(viz, (x,y), 1, (0,0,0), -1)
    
    for label, typed_polygons in landmark_polygon_vertices_global.iteritems():
        for ptype, vs in typed_polygons:
            pts_canvas = vs + origin_on_canvas
            
#             for x,y in pts_canvas.astype(np.int):
#                 cv2.circle(viz, (x,y), 5, colors[label], -1)

            if ptype == PolygonType.OPEN or ptype == PolygonType.DIRECTION:
                cv2.polylines(viz, [pts_canvas.astype(np.int)], False, colors[label], 5)
            else:
                cv2.polylines(viz, [pts_canvas.astype(np.int)], True, colors[label], 5)

                
    return viz

In [22]:
labeling_name = 'RS141_0001_yuncong_08282015013756'

In [23]:
with open('/home/yuncong/csd395/landmark_descriptors/%s_landmark_polygon_vertices_global.pkl'%labeling_name, 'r') as f:
    landmark_polygon_vertices_global = pickle.load(f)

In [34]:
atlas_viz = visualize_atlas(landmark_polygon_vertices_global)
display(atlas_viz)

/oasis/projects/nsf/csd395/yuncong/Brain/notebooks/tmp.jpg

In [27]:
imsave(os.environ['GORDON_RESULT_DIR'] + '/atlas_viz.jpg', atlas_viz)

In [28]:
atlas_origin_on_canvas = DataManager(generate_hierarchy=False, stack='RS141', resol='x5', section=1).load_pipeline_result('originPosOnCanvas','npy')
np.save(os.environ['GORDON_RESULT_DIR'] + '/atlas_origin_on_canvas.npy', atlas_origin_on_canvas)